# Quickstart Guide for Strands Agents

Strands Agents is a powerful framework for building AI agents that can interact with AWS services and perform complex tasks. This quickstart guide will help you get started with creating your first Strands agent.

## Prerequisites

- Python 3.10 or later
- AWS account configured with appropriate permissions
- Basic understanding of Python programming

Lets get started !

In [1]:
# Install Strands using pip

!pip install strands-agents strands-agents-tools

  Using cached certifi-2025.11.12-py3-none-any.whl.metadata (2.5 kB)
  Using cached cryptography-46.0.3-cp311-abi3-manylinux_2_34_x86_64.whl.metadata (5.7 kB)
  Using cached cffi-2.0.0-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (2.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 29.7 MB/s  0:00:00m0:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 43.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 45.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 40.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 39.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 21.9 MB/s  0:00:00
Using cached certifi-2025.11.12-py3-none-any.whl (159 kB)
Using cached cryptography-46.0.3-cp311-abi3-manylinux_2_34_x86_64.whl (4.5 MB)
Using cached cffi-2.0.0-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (219 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69/

In [17]:
!pip install dotenv
!pip install litellm
!pip install cohere


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 15.9 MB/s  0:00:00m0:00:01
  Attempting uninstall: httpx-sse
    Found existing installation: httpx-sse 0.4.3
    Uninstalling httpx-sse-0.4.3:
      Successfully uninstalled httpx-sse-0.4.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [cohere]2m3/4 [cohere]o]]


In [18]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [32]:
import os
from dotenv import load_dotenv
from strands import Agent, tool
from strands_tools import calculator
import litellm
load_dotenv()

True

In [34]:
# OpenRouter gives you Claude, Gemini, Cohere, etc. via BedrockModel interface
litellm.api_key = os.getenv('OPENROUTER_API_KEY')  # Get from openrouter.ai
litellm.base_url = "https://openrouter.ai/api/v1"


agent = Agent(
    model="anthropic/claude-3.5-sonnet",  # Works via LiteLLM proxy
    tools=[],
    system_prompt="Voice AI for developer Soham."
)



In [35]:
print(agent("42*7 and greet me"))

NoCredentialsError: Unable to locate credentials

In [28]:
import os
from cohere import Client
from strands import Agent, tool
from strands_tools import calculator

class CohereModel:
    def __init__(self, model_id, api_key):
        self.client = Client(api_key)
        self.model_id = model_id
    
    def stream(self, messages, **kwargs):  # ← FIXED: Added **kwargs
        """Strands calls: model.stream(messages, **kwargs)"""
        # Extract content from messages (list of dicts)
        content = ""
        if isinstance(messages, list):
            for msg in messages:
                if isinstance(msg, dict) and 'content' in msg:
                    content += msg['content'] + "\n"
                elif hasattr(msg, 'content'):
                    content += msg.content + "\n"
        
        stream = self.client.chat_stream(
            content, 
            model=self.model_id, 
            temperature=0.1,
            **kwargs  # Pass through Strands kwargs
        )
        
        for chunk in stream:
            # Yield format Strands expects
            yield {"content": chunk.text}
    
    def complete(self, messages, **kwargs):  # ← FIXED: Added **kwargs
        content = ""
        if isinstance(messages, list):
            for msg in messages:
                if isinstance(msg, dict) and 'content' in msg:
                    content += msg['content'] + "\n"
                elif hasattr(msg, 'content'):
                    content += msg.content + "\n"
        
        response = self.client.chat(
            content, 
            model=self.model_id,
            **kwargs
        )
        return {"content": response.text}


In [29]:
model = CohereModel(
    model_id="command-a-08-2024",
    api_key=os.getenv('COHERE_API_KEY')
)

In [30]:
agent = Agent(
    model=model,
    tools=[],
    system_prompt="Voice AI assistant."
)

In [31]:
agent("What's 100/4? Who built me?")


TypeError: CohereModel.stream() takes 2 positional arguments but 4 were given

In [23]:
agent = Agent(
    model=model,
    tools=[],
    system_prompt="Voice AI assistant."
)

In [9]:
print("\n=== MODELS MODULE ===")
import strands.models
print([m for m in dir(strands.models) if not m.startswith('_')])


=== MODELS MODULE ===
['BedrockModel', 'Model', 'bedrock', 'model']


In [8]:
from strands.models.cohere import Cohere  # ← Correct import
model = Cohere(
    model="command-a-08-2024",
    api_key=os.getenv('COHERE_API_KEY')
)

ModuleNotFoundError: No module named 'strands.models.cohere'

In [6]:
from strands import Agent, tool
from strands_tools import calculator
import os
# Direct model config (no separate model class needed)
os.environ["COHERE_API_KEY"] = os.getenv('COHERE_API_KEY')

agent = Agent(
    model={
        "provider": "cohere",
        "model": "command-a-08-2024",
        "api_key": os.getenv('COHERE_API_KEY'),  # Pass directly
        "no_aws": True  # Disable Bedrock fallback
    },
    # tools=[calculator, voice_context],
    system_prompt="Voice AI assistant. Be concise and helpful."
)



In [36]:
import os
from dotenv import load_dotenv
from strands import Agent, tool
from strands_tools import calculator
from cohere import Client

load_dotenv()

class PerfectCohereModel:
    """100% Strands-compatible Cohere model - NO AWS DEPENDENCY"""
    
    def __init__(self, model_id="command-a-08-2024", api_key=None):
        self.client = Client(api_key or os.getenv('COHERE_API_KEY'))
        self.model_id = model_id
    
    def stream(self, messages, **kwargs):
        """EXACT Strands signature - handles ALL edge cases"""
        # Handle different messages formats Strands sends
        if isinstance(messages, str):
            content = messages
        elif hasattr(messages, 'content'):
            content = messages.content
        elif isinstance(messages, list) and len(messages) > 0:
            content = messages[-1].get('content', str(messages[-1]))
        else:
            content = str(messages)
        
        stream = self.client.chat_stream(
            content.strip(),
            model=self.model_id,
            temperature=kwargs.get('temperature', 0.1),
            max_tokens=kwargs.get('max_tokens', 512)
        )
        
        for chunk in stream:
            yield type('obj', (object,), {
                'content': getattr(chunk, 'text', str(chunk)),
                'delta': getattr(chunk, 'text', '')
            })()
    
    def complete(self, messages, **kwargs):
        """Sync fallback"""
        content = str(messages).strip()
        response = self.client.chat(
            content,
            model=self.model_id,
            temperature=kwargs.get('temperature', 0.1)
        )
        return type('obj', (object,), {'content': response.text})()


In [ ]:

# CREATE MODEL (No AWS touch this)
model = PerfectCohereModel(api_key=os.getenv('COHERE_API_KEY'))


# CREATE AGENT (Pure custom model - no Bedrock)
agent = Agent(
    model=model,
    tools=[],
    system_prompt="Voice AI for AI engineer Soham. Concise, technical."
)

# TEST - THIS WILL WORK 100%
print(agent("What's 100/4? What's my framework?"))

TypeError: PerfectCohereModel.stream() takes 2 positional arguments but 4 were given

In [7]:
output = agent('Tell one joke')

AttributeError: 'dict' object has no attribute 'stream'


## Creating Your First Agent

Lets get an overview of the agentic components needed.

### 1. Create a simple Agent:

This will create an agent with the default model provider, [Amazon Bedrock](https://aws.amazon.com/bedrock/), and the default model, Claude 3.7 Sonnet, in the region of your AWS setup. While the agent runs in the same local environment as it is being invoked, Amazon Bedrock models will run in an AWS account and your agent will invoke the model in the cloud account. The architecture looks as following:

<div style="text-align:center">
    <img src="images/simple_agent.png" width="75%" />
</div>

In [ ]:

import warnings
warnings.filterwarnings(action="ignore", message=r"datetime.datetime.utcnow") 

from strands import Agent
# Initialize your agent
# agent = Agent(
    # model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
    # system_prompt="You are a helpful assistant that provides concise responses."
# )

agent = Agent(
    model=cohere_model,
    system_prompt="You are a helpful assistant that provides concise responses."
)

# Send a message to the agent
response = agent("Hello! Tell me a joke.")

### 2. Add Tools to the Agent:

The [strands-agents-tools](https://github.com/strands-agents/tools) repository provides some in-built tools which you can import. You can also create custom tools using the `@tool` decorator. We can create agents with built-in and custom tools. For instance, adding the built-in tool of a calculator and a custom tool for getting the weather you get the following architecture:
<div style="text-align:center">
    <img src="images/agent_with_tools.png" width="75%" />
</div>

Implementing this architecture you have the following:

In [ ]:
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"

agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather.")

response = agent("What is the weather today?")
print(response)

In [39]:
import os
from dotenv import load_dotenv
from strands import Agent, tool
from strands_tools import calculator

load_dotenv()

# Block AWS
os.environ["AWS_ACCESS_KEY_ID"] = "none"
os.environ["AWS_SECRET_ACCESS_KEY"] = "none"

# ✅ SET BOTH (Covers all possibilities)
cohere_key = os.getenv('COHERE_API_KEY')
os.environ["COHERE_API_KEY"] = cohere_key
os.environ["OPENAI_API_KEY"] = cohere_key  # OpenAI-compatible endpoint uses this too

@tool
def soham_context() -> dict:
    return {
        "name": "Soham",
        "framework": "Pipecat + Strands",
        "goal": "Agentic voice AI"
    }

agent = Agent(
    model="cohere/command-a-08-2024",
    tools=[calculator, soham_context],
    system_prompt="Voice AI for developer Soham. Concise, technical."
)

result = agent("What's 42*7? Who am I building for?")
print(result.message)


ClientError: An error occurred (UnrecognizedClientException) when calling the ConverseStream operation: The security token included in the request is invalid.

### Invoking tool directly

For some applications it is important to directly call the tool. For instance, you might want to debug the tool, pre-populate the agent knowledge with your customer's information or using a tool inside of another tool. In Strands you can do it using the ``tool`` method of your agent followed by the tool name

In [ ]:
# Alternatively, you can invoke the tool directly like so:
agent.tool.calculator(expression="sin(x)", mode="derive", wrt="x", order=2)

In [ ]:
import os
from dotenv import load_dotenv
from cohere import Client
from strands import Agent, tool
from strands_tools import calculator

load_dotenv()

# ✅ CUSTOM COHERE MODEL (ACCEPTS YOUR API KEY)
class CohereModel:
    def __init__(self, api_key: str, model_id: str = "command-a-08-2024"):
        self.client = Client(api_key=api_key)
        self.model_id = model_id
    
    async def stream(self, messages, tool_specs=None, system_prompt=None, **kwargs):
        """Strands-compatible streaming"""
        content = ""
        if isinstance(messages, list):
            for msg in messages:
                if isinstance(msg, dict):
                    content += msg.get('content', '') + "\n"
        
        stream = self.client.chat_stream(
            message=content.strip(),
            model=self.model_id,
            temperature=0.1
        )
        
        for chunk in stream:
            text = getattr(chunk, 'text', '')
            if text:
                yield {"contentBlockDelta": {"delta": {"text": text}}}
        
        yield {"type": "message_stop", "stop": {"stop_reason": "end_turn"}}
    
    def complete(self, messages, **kwargs):
        """Sync fallback"""
        content = ""
        if isinstance(messages, list):
            for msg in messages:
                content += msg.get('content', '') + "\n"
        
        response = self.client.chat(message=content.strip(), model=self.model_id)
        return {"stop": {"stop_reason": "end_turn", "message": {"content": response.text}}}


# ✅ CREATE MODEL WITH YOUR API KEY (EXPLICIT)
cohere_model = CohereModel(api_key=os.getenv('COHERE_API_KEY'))

@tool
def soham_context() -> dict:
    return {"name": "Soham", "framework": "Pipecat + Strands"}

# ✅ CREATE AGENT (Model object, not string)
agent = Agent(
    model=cohere_model,  # ← Your custom model with your API key
    tools=[calculator, soham_context],
    system_prompt="Voice AI for Soham."
)

# TESTimport os
from dotenv import load_dotenv
from cohere import Client
from strands import Agent, tool
from strands_tools import calculator

load_dotenv()

# ✅ FIXED COHERE MODEL
class CohereModel:
    def __init__(self, api_key: str, model_id: str = "command-a-08-2024"):
        self.client = Client(api_key=api_key)
        self.model_id = model_id
    
    async def stream(self, messages, tool_specs=None, system_prompt=None, **kwargs):
        """Strands-compatible streaming"""
        content = ""
        if isinstance(messages, list):
            for msg in messages:
                if isinstance(msg, dict):
                    content += msg.get('content', '') + "\n"
        
        stream = self.client.chat_stream(
            message=content.strip(),
            model=self.model_id,
            temperature=0.1
        )
        
        for chunk in stream:
            # ✅ FIX: Handle both list and string formats from Cohere
            text = chunk.text
            if isinstance(text, list):
                text = "".join(text) if text else ""
            
            if text:
                yield {"contentBlockDelta": {"delta": {"text": text}}}
        
        yield {"type": "message_stop", "stop": {"stop_reason": "end_turn"}}
    
    def complete(self, messages, **kwargs):
        content = ""
        if isinstance(messages, list):
            for msg in messages:
                content += msg.get('content', '') + "\n"
        
        response = self.client.chat(message=content.strip(), model=self.model_id)
        return {"stop": {"stop_reason": "end_turn", "message": {"content": response.text}}}


# ✅ Your API key goes here
cohere_model = CohereModel(api_key=os.getenv('COHERE_API_KEY'))

@tool
def soham_context() -> dict:
    return {"name": "Soham", "framework": "Pipecat + Strands"}

agent = Agent(
    model=cohere_model,
    tools=[calculator, soham_context],
    system_prompt="Voice AI for Soham."
)

print(agent)
agent.
# # TEST
# result = agent(["What's 42*7? Who am I?"])
# print(result.message)




### 3. Changing the log level and format:

Strands SDK uses Python's standard `logging` module to provide visibility into its operations.

The Strands Agents SDK implements a straightforward logging approach:

1. **Module-level Loggers**: Each module in the SDK creates its own logger using logging.getLogger(__name__), following Python best practices for hierarchical logging.
2. **Root Logger**: All loggers in the SDK are children of the "strands" root logger, making it easy to configure logging for the entire SDK.
3. **Default Behavior**: By default, the SDK doesn't configure any handlers or log levels, allowing you to integrate it with your application's logging configuration.

To enable logging for the Strands Agents SDK, you can configure the **"strands"** logger. If you want to change the log level, for example during debugging, or modify the log format, you can set the logger configuration as follows:

In [ ]:
import logging
from strands import Agent

# Enables Strands debug log level
logging.getLogger("strands").setLevel(logging.DEBUG) # or logging.INFO

# Sets the logging format and streams logs to stderr
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)

agent = Agent(model="us.anthropic.claude-3-7-sonnet-20250219-v1:0")  # Optional: Specify the model ID
agent("Hello!")


### 4. Model Provider

The default model provider is [Amazon Bedrock](https://aws.amazon.com/bedrock/) and the default model is Claude 3.7 Sonnet in the region of your current AWS environment

You can specify a different model in Amazon Bedrock providing the model ID string directly:

In [ ]:
from strands import Agent

agent = Agent(model="anthropic.claude-3-5-haiku-20241022-v1:0")
print(agent.model.config)


For more control over the model configuration, you can create a `BedrockModel` provider instance:

In [ ]:
import boto3
from strands import Agent
from strands.models import BedrockModel

# Create a BedrockModel
bedrock_model = BedrockModel(
    model_id="anthropic.claude-3-5-haiku-20241022-v1:0",
    region_name='us-west-2',
    temperature=0.3,
)

agent = Agent(model=bedrock_model)


More details on the available model providers on the [Model Provider Quickstart page](https://strandsagents.com/0.1.x/user-guide/quickstart/#model-providers)


**Congratulations !! Now you have learned how to build a simple agent using Strands!!**

## [Optional] Lets Build a Task-Specific Agent - RecipeBot 🍽️

Lets create a practical example of a task-specific agent. We create a `RecipeBot` that recommends recipes and answers any cooking related questions. Lets dive in !!

Here's what we will create :

<div style="text-align:center">
    <img src="images/interactive_recipe_agent.png" width="75%" />
</div>

In [ ]:
# Install the required packages
%pip install ddgs # Also install strands-agents strands-agents-tools if you haven't already

In [ ]:
from strands import Agent, tool
from ddgs import DDGS
from ddgs.exceptions import RatelimitException, DDGSException
import logging

# Configure logging
logging.getLogger("strands").setLevel(logging.INFO)

# Define a websearch tool
@tool
def websearch(keywords: str, region: str = "us-en", max_results: int | None = None) -> str:
    """Search the web to get updated information.
    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "RatelimitException: Please try again after a short delay."
    except DDGSException as d:
        return f"DuckDuckGoSearchException: {d}"
    except Exception as e:
        return f"Exception: {e}"


In [ ]:
# Create a recipe assistant agent
recipe_agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
    system_prompt="""You are RecipeBot, a helpful cooking assistant.
    Help users find recipes based on ingredients and answer cooking questions.
    Use the websearch tool to find recipes when users mention ingredients or to
    look up cooking information.""",
    # Import the websearch tool we created above
    tools=[websearch],
)

In [ ]:
response = recipe_agent("Suggest a recipe with chicken and broccoli.")

# Other examples:
# response = recipe_agent("How do I cook quinoa?")
# response = recipe_agent("How can I substitute white wine in shrimp pasta?")
# response = recipe_agent("What are the health benefits of asparagus?")
print(response)

#### And that's it! We now have a running usecase agent with tools in just a few lines of code 🥳.

For more detailed quickstart guide, check out the [Strands documentation](https://strandsagents.com/0.1.x/user-guide/quickstart/).

### [Optional] Run RecipeBot via CLI: 
you can run the agent in interactive mode via the command line (for instance using the terminal on SageMaker Studio) through the python script provided in `02_simple_interactive_usecase/recipe_bot.py`. This allows you to interact with the agent in a more dynamic way, sending messages and receiving responses via the CLI.
Run these commands on a command line interface to run the agent in interactive mode:

```cli
cd samples/01-tutorials/01-fundamentals/01-first-agent/02-simple-interactive-usecase/
pip install -r requirements.txt
python recipe_bot.py
```

With this, you can talk to the bot via a command line interface(CLI).